In [1]:
using Gen

Let's dig into what exact inference looks like for multi-object tracking. 

Let's imagine a very simple model -- between 1-4 objects spawn on a 5x5 grid. What's the initial probability of a starting condition?

The probability of $N$ objects being sampled is $\frac{1}{4}$. For any given object, the probability it appears in any specific location on the grid is proportional to the scene size. Since we sample both $x$ and $y$ coordinates independently that probability is $\frac{1}{25}$. 

Now, for a scene with 4 objects, the joint probability of sampling 4 objects at 4 specific locations is: $(\frac{1}{4})(\frac{1}{25})^4$, or $6e-7$. A pretty low likelihood for any given configuration. 

We can illustrate that below with Gen:

In [124]:
@gen function sample_position(scene_size::Int64)
    """
    Return a 
    """
    x = {:x} ~ uniform_discrete(1, scene_size)
    y = {:y} ~ uniform_discrete(1, scene_size)
    return x, y
end

@gen function init_state(max_objects::Int64, scene_size::Int64)
    """
    Generative model for initial positions of an unknown number of objects in a scene
    """
    n = {:n} ~ uniform_discrete(1, max_objects)
    get_positions = Map(sample_position)
    sizes = [scene_size for i in 1:n]
    positions ~ get_positions(sizes)
    return positions
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Int64, Int64], false, Union{Nothing, Some{Any}}[nothing, nothing], var"##init_state#350", Bool[0, 0], false)

In [141]:
tr = simulate(init_state, (4, 5))
choices = get_choices(tr)
println(choices[:positions => 1 => :y])
println(exp(get_score(tr)))

3
0.0004000000000000001


In [166]:
tr, w = generate(init_state, (4, 5), choicemap((:positions => 4 => :y, 10)))
println(w)

tr, w = generate(init_state, (4, 5), choicemap((:n, 4)))
println(exp(w))

0.0
0.25
